In [251]:
from dotenv import load_dotenv
from openai import OpenAI
import gradio as gr
import json
import os

In [252]:
load_dotenv()
api_key = os.getenv("OPENAI_API_KEY")

if api_key:
    if api_key.startswith('sk-proj'):
        print("OpneAI API Key Loaded")
    else:
        print("Non Proper OpenAI api key")
else:
    print("Did not Find API Key")

OpneAI API Key Loaded


In [253]:
openai = OpenAI()

## **Function Tools**

In [254]:
def get_price(city):
    price_dict = {'tokyo':'1400','france':'900','rome':'400'}

    return price_dict.get(city.lower(),"we do not have tickets for that city yet!")

In [255]:
tickets = {'tokyo':5,'france':5,'rome':5,'spain':4}

def get_bookings(city,no_of_tickets):
    temp = tickets.get(city,None)
    if temp and temp >= no_of_tickets and temp != 0:
        tickets[city] = tickets[city] - no_of_tickets
        return "Booking Successful"
    return "Booking Unsuccessful"

In [256]:
tools = [
{
    "type": "function",
    "function": {
        "name": "get_price",
        "description": "Get the price to destination city.",
        "parameters": {
            "type": "object",
            "properties": {
                "city": {
                    "type": "string",
                    "description": "the destination city user wants price of"
                }
            },
            "required": [
                "city"
            ],
            "additionalProperties": False
        },
        "strict": True
    }
},
{
    "type": "function",
    "function": {
        "name": "get_bookings",
        "description": "Get the booking done if tickets are available for destination city",
        "parameters": {
            "type": "object",
            "properties": {
                "city": {
                    "type": "string",
                    "description": "the destination city user wants price of"
                },
                "no_of_tickets": {
                "type": "integer",
                "description": "the amount of tickets user want to book for that destination"
                }
            },
            "required": [
                "city",
                "no_of_tickets"
            ],
            "additionalProperties": False
        },
        "strict": True
    }
}]

## **How tool calls work**

In [208]:
result = openai.chat.completions.create(
    model="gpt-4o-mini",
    messages=[{"role": "user", "content": "What is the price to tokyo?"}],
    tools=tools
)

In [209]:
# Print the full result message returned by the model
print("Full model response:", result.choices[0])

# Print the assitant message returned by the model
print("\nAssistant response:", result.choices[0].message)

# Print the finish reason for the response
print("\nFinish reason:", result.choices[0].finish_reason)

# Extract the tool call from the model's result response
tool_call = result.choices[0].message.tool_calls[0]  # Access the first tool call

# Print the tool call made by the model
print("\nTool call details:", tool_call)

# Parse the arguments (JSON string to dictionary)
args = json.loads(tool_call.function.arguments)  # Convert JSON string to dictionary

# Access the name of the tool/function being called
function_name = tool_call.function.name  # Extract tool name

# Print tool name and parsed arguments
print("\nTool name:", function_name)
print("Tool call arguments:", args)

Full model response: Choice(finish_reason='tool_calls', index=0, logprobs=None, message=ChatCompletionMessage(content=None, refusal=None, role='assistant', audio=None, function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_dN1W5gLpN8i5wi79JDUGSYyF', function=Function(arguments='{"city":"Tokyo"}', name='get_price'), type='function')]))

Assistant response: ChatCompletionMessage(content=None, refusal=None, role='assistant', audio=None, function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_dN1W5gLpN8i5wi79JDUGSYyF', function=Function(arguments='{"city":"Tokyo"}', name='get_price'), type='function')])

Finish reason: tool_calls

Tool call details: ChatCompletionMessageToolCall(id='call_dN1W5gLpN8i5wi79JDUGSYyF', function=Function(arguments='{"city":"Tokyo"}', name='get_price'), type='function')

Tool name: get_price
Tool call arguments: {'city': 'Tokyo'}


## **Making tool Calls**

In [257]:
def handle_tool_call(message):
    
    tool_asked = message.tool_calls[0].function.name
    arguments = json.loads(message.tool_calls[0].function.arguments) # str --> json 
    
    if tool_asked == 'get_price':
        result = get_price(arguments['city'])
        
    elif tool_asked == 'get_bookings':
        result = get_bookings(arguments['city'],arguments['no_of_tickets'])

    response = {"role": "tool",
                "content": result,
                "tool_call_id": message.tool_calls[0].id}
    
    return response

In [258]:

system_message = "You are a helpful assistant for an Airline called FlightAI. "
system_message += "Give short, courteous answers, no more than 1 sentence. "
system_message += "Always be accurate. If you don't know the answer, say so."

def call_gpt_with_tools(user_msg,history):

    messages = [{'role':'system','content':system_message}] + history + [{'role':'user','content':user_msg}]

    result = openai.chat.completions.create(
        model="gpt-4o-mini",
        messages=messages,
        tools=tools
    )
    
    # Making tool calls 
    if result.choices[0].finish_reason == 'tool_calls':
        message = result.choices[0].message
        print(message)
        response = handle_tool_call(message)
        
        messages.append(message)
        messages.append(response)

        result = openai.chat.completions.create(model='gpt-4o-mini',messages=messages)

    return result.choices[0].message.content

In [259]:
gr.ChatInterface(fn=call_gpt_with_tools,type='messages').launch(debug=True) 

* Running on local URL:  http://127.0.0.1:7870

To create a public link, set `share=True` in `launch()`.


ChatCompletionMessage(content=None, refusal=None, role='assistant', audio=None, function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_yodBU65vXPWVQv7ALayOmWwt', function=Function(arguments='{"city":"rome"}', name='get_price'), type='function')])
ChatCompletionMessage(content=None, refusal=None, role='assistant', audio=None, function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_HJrSwR0R823QQvHMWluFeYbc', function=Function(arguments='{"city":"rome","no_of_tickets":3}', name='get_bookings'), type='function')])
ChatCompletionMessage(content=None, refusal=None, role='assistant', audio=None, function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_k1fikhmYDFJYyeFUGPENSvyf', function=Function(arguments='{"city":"rome","no_of_tickets":3}', name='get_bookings'), type='function')])
Keyboard interruption in main thread... closing server.
